<a href="https://colab.research.google.com/github/KayvanShah1/usc-csci-544-assignments-hw/blob/main/hw1/HW1_CSCI544.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

## Install

In [3]:
!pip install contractions
!pip install ipython-autotime

## Imports

In [37]:
import os
import re
import unicodedata

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import contractions

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score

from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 11.4 ms (started: 2023-09-12 13:07:35 +00:00)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Dataset Preparation

## Download Data

In [5]:
# %%bash
# cd "/content/drive/MyDrive/Colab Notebooks/CSCI544/HW1"
# curl -o amazon_reviews_us_Office_Products_v1_00.tsv.gz \
# https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz

time: 344 µs (started: 2023-09-12 12:44:38 +00:00)


## Read Data

- `sep='\t'`: Values in the TSV file are separated by tabs

- `on_bad_lines="skip"`: Skip any lines in the file that are improperly formatted or contain errors.

- `memory_map=True`: Maps the file obj directly to memory for direct access improving performance for large files

- `nrows=10`: Limits the number of rows to read from the file to 10.

- `usecols=["review_headline","review_body", "star_rating"]`: Only select subset of columns to read - faster parsing time and low memory usage.

In [6]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/CSCI544/HW1")
DATA_PATH = "amazon_reviews_us_Office_Products_v1_00.tsv.gz"

time: 6.95 ms (started: 2023-09-12 12:44:38 +00:00)


Have quick look at dataset by reading 10 rows to get the column names



In [7]:
df_small = pd.read_csv(
    DATA_PATH,
    sep='\t',
    on_bad_lines="skip",
    memory_map=True,
    nrows=10
)
df_small.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

time: 56.2 ms (started: 2023-09-12 12:44:38 +00:00)


In [8]:
df_small.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,"Scotch Cushion Wrap 7961, 12 Inches x 100 Feet",Office Products,5,0,0,N,Y,Five Stars,Great product.,2015-08-31
1,US,10951564,R3L4L6LW1PUOFY,B00DZYEXPQ,75004341,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0,1,N,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31
2,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0,0,N,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31
3,US,52782374,R1PR37BR7G3M6A,B00D7H8XB6,868449945,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2,3,N,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31
4,US,24045652,R3BDDDZMZBZDPU,B001XCWP34,33521401,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0,0,N,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31


time: 63.2 ms (started: 2023-09-12 12:44:38 +00:00)


Read the entire data

In [9]:
df = pd.read_csv(
    DATA_PATH,
    sep='\t',
    usecols=["review_headline","review_body", "star_rating"],
    on_bad_lines="skip",
    memory_map=True,
)

<ipython-input-9-6e7d86bcb04e>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


time: 55.1 s (started: 2023-09-12 12:44:38 +00:00)


In [10]:
df.head()

,star_rating,review_headline,review_body
0,5,Five Stars,Great product.
1,5,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...
2,5,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it."
3,1,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...
4,4,Four Stars,Gorgeous colors and easy to use


time: 10.3 ms (started: 2023-09-12 12:45:33 +00:00)


## Keep Reviews and Ratings

In [11]:
df_filtered = df.loc[:,['review_body', 'star_rating']]
df_filtered.head()

,review_body,star_rating
0,Great product.,5
1,What's to say about this commodity item except...,5
2,"Haven't used yet, but I am sure I will like it.",5
3,Although this was labeled as &#34;new&#34; the...,1
4,Gorgeous colors and easy to use,4


time: 138 ms (started: 2023-09-12 12:45:33 +00:00)


## Create Binary Classification Problem
We form two classes and select 50000 reviews randomly from each class.

In [12]:
df['star_rating'].unique()

array([5, 1, 4, 2, 3, '5', '1', '3', '4', '2', '2015-06-05', '2015-02-11',
       nan, '2014-02-14'], dtype=object)

time: 109 ms (started: 2023-09-12 12:45:34 +00:00)


In [13]:
# Convert the 'star_rating' column to numeric, coerce errors to NaN
df_filtered['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')

# Drop NaN values from `star_rating`
df_filtered.dropna(subset=["star_rating"], inplace=True)

# Classify ratings as 1, 2, or 3 into class 1, and ratings 4 and 5 into class 2
df_filtered['sentiment'] = df_filtered['star_rating'].apply(
    lambda x: 1 if x <= 3 else 2
)

time: 4.54 s (started: 2023-09-12 12:45:34 +00:00)


In [14]:
df_filtered.head()

,review_body,star_rating,sentiment
0,Great product.,5.0,2
1,What's to say about this commodity item except...,5.0,2
2,"Haven't used yet, but I am sure I will like it.",5.0,2
3,Although this was labeled as &#34;new&#34; the...,1.0,1
4,Gorgeous colors and easy to use,4.0,2


time: 9.72 ms (started: 2023-09-12 12:45:38 +00:00)


In [15]:
print("Shape of unfiltered dataframe:", df.shape)
print("Shape of filtered dataframe:", df_filtered.shape)

Shape of unfiltered dataframe: (2640352, 3)
Shape of filtered dataframe: (2640335, 3)
time: 4.01 ms (started: 2023-09-12 12:45:38 +00:00)


### Sampling data
- Find indices of each class
- Choose random 50000 values using `sample` function
- Resample for shuffling

In [16]:
RANDOM_STATE = 56

# Create a new DataFrame with sampled data
balanced_df = pd.concat(
    [
        df_filtered.query('sentiment==1').sample(
            n=50000, random_state=RANDOM_STATE
        ),
        df_filtered.query('sentiment==2').sample(
            n=50000, random_state=RANDOM_STATE
        )
    ],
    ignore_index=True
).sample(frac=1, random_state=RANDOM_STATE)

balanced_df.drop(columns=["star_rating"], inplace=True)

# Handling non-string values in Reviews
balanced_df["review_body"] = balanced_df["review_body"].astype(str)

balanced_df.head()

,review_body,sentiment
7527,Agree with other posters in that these worked ...,1
84247,These are a little smaller than the ones I had...,2
79106,"Well made, plenty of card pockets, and I like ...",2
37339,"In the same year, I bought a new HP laptop and...",1
33018,I bought this on Jan 2011 as a gift. I turned ...,1


time: 513 ms (started: 2023-09-12 12:45:38 +00:00)


In [17]:
del df_small
del df

time: 531 µs (started: 2023-09-12 12:45:39 +00:00)


# Data Cleaning

References:

[1] https://www.kaggle.com/code/kayvanshah/roberta-large-2?scriptVersionId=34903957&cellId=11

In [18]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def expand_contractions(text):
    return contractions.fix(text)

def clean_text(text):
    text = unicode_to_ascii(text.lower().strip())

    # replacing email addresses with empty string
    text = re.sub(
        r"[a-zA-Z0-9_\-\.]+@[a-zA-Z0-9_\-\.]+\.[a-zA-Z]{2,5}", " ", text
    )

    # replacing urls with blank empty string
    text = re.sub(
        r"\bhttp:\/\/([^\/]*)\/([^\s]*)|https:\/\/([^\/]*)\/([^\s]*)", " ", text
    )

    # Remove HTML tags with empty string
    text = re.sub(r"<.*?>", "", text)

    # Expand contraction for eg., wouldn't => would not
    text = expand_contractions(text)

    # creating a space between a word and the punctuation following it
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)

    # removes all non-alphabetical characters
    text = re.sub(r"[^a-zA-Z\s]+", "", text)

    # remove extra spaces
    text = re.sub(" +", " ", text)

    text = text.strip()
    return text

clean_text_vect = np.vectorize(clean_text)

time: 2.63 ms (started: 2023-09-12 12:45:39 +00:00)


In [19]:
# Calculate average length of reviews before cleaning (combined)
average_length_before_clean = balanced_df["review_body"].apply(len).mean()

balanced_df["review_body"] = balanced_df["review_body"].apply(clean_text_vect)

# Calculate average length of reviews after cleaning (combined)
average_length_after_clean = balanced_df["review_body"].apply(len).mean()

print(f'Average Length of Reviews Before Cleaning (Combined): {average_length_before_clean:.2f} characters')
print(f'Average Length of Reviews After Cleaning (Combined): {average_length_after_clean:.2f} characters')

Average Length of Reviews Before Cleaning (Combined): 314.91 characters
Average Length of Reviews After Cleaning (Combined): 299.72 characters
time: 49.9 s (started: 2023-09-12 12:45:39 +00:00)


# Pre-processing

In [20]:
# Stopword list
pattern = re.compile(r'\b('+r'|'.join(stopwords.words('english'))+r')\b\s*')

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

def preprocess_text(text):
    # replacing all the stopwords
    text = pattern.sub('',text)
    text = lemmatize_text(text)
    return text

preprocess_text_vect = np.vectorize(preprocess_text)

time: 6.11 ms (started: 2023-09-12 12:46:29 +00:00)


In [21]:
# Calculate average length of reviews before cleaning (combined)
average_length_before_preprocess = balanced_df["review_body"].apply(len).mean()

balanced_df["review_body"] = balanced_df["review_body"].apply(preprocess_text_vect)

# Calculate average length of reviews after cleaning (combined)
average_length_after_preprocess = balanced_df["review_body"].apply(len).mean()

print(f'Average Length of Reviews Before Preprocessing: {average_length_before_preprocess:.2f} characters')
print(f'Average Length of Reviews After Preprocessing: {average_length_after_preprocess:.2f} characters')

Average Length of Reviews Before Preprocessing: 299.72 characters
Average Length of Reviews After Preprocessing: 185.34 characters
time: 1min 29s (started: 2023-09-12 12:46:29 +00:00)


### Train and Test Split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    balanced_df['review_body'],
    balanced_df['sentiment'],
    test_size=0.2,
    random_state=RANDOM_STATE
)

time: 16.4 ms (started: 2023-09-12 12:47:59 +00:00)


In [23]:
del df_filtered
del balanced_df

time: 563 µs (started: 2023-09-12 12:47:59 +00:00)


# Feature Extraction

## TF-IDF

In [24]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

time: 4.43 s (started: 2023-09-12 12:47:59 +00:00)


## Bag of words

In [25]:
count_vectorizer = CountVectorizer(max_features=5000)
X_train_bow = count_vectorizer.fit_transform(X_train)
X_test_bow = count_vectorizer.transform(X_test)

time: 3.05 s (started: 2023-09-12 12:48:03 +00:00)


# ML Modeling

In [26]:
def evaluate_model(model, X_test, y_test):
    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate evaluation metrics
    precision = precision_score(y_test, y_pred, average='binary')
    recall = recall_score(y_test, y_pred, average='binary')
    f1 = f1_score(y_test, y_pred, average='binary')

    return precision, recall, f1

time: 911 µs (started: 2023-09-12 12:48:38 +00:00)


## Perceptron Using Both Features

In [27]:
def train_evaluate_perceptron(X_train, y_train, X_test, y_test):
    # Initialize Perceptron model
    perceptron = Perceptron(max_iter=4000)

    # Train the model
    perceptron.fit(X_train, y_train)

    # Evaluate model
    precision, recall, f1 = evaluate_model(perceptron, X_test, y_test)
    return precision, recall, f1


# Train and evaluate Perceptron model using BoW features
precision_perceptron_bow, recall_perceptron_bow, f1_perceptron_bow = train_evaluate_perceptron(X_train_bow, y_train, X_test_bow, y_test)

# Train and evaluate Perceptron model using TF-IDF features
precision_perceptron_tfidf, recall_perceptron_tfidf, f1_perceptron_tfidf = train_evaluate_perceptron(X_train_tfidf, y_train, X_test_tfidf, y_test)

# Print the results
print(f'Precision Recall F1 (BoW): {precision_perceptron_bow:.2f} {recall_perceptron_bow:.2f} {f1_perceptron_bow:.2f}')
print(f'Precision Recall F1 (TF-IDF): {precision_perceptron_tfidf:.2f} {recall_perceptron_tfidf:.2f} {f1_perceptron_tfidf:.2f}')

Precision Recall F1 (BoW): 0.82 0.76 0.79
Precision Recall F1 (TF-IDF): 0.78 0.81 0.80
time: 541 ms (started: 2023-09-12 12:48:43 +00:00)


## SVM Using Both Features

In [38]:
def train_evaluate_svm(X_train, y_train, X_test, y_test):
    # Initialize SVM model
    svm = LinearSVC(max_iter=3500)

    # Train the model
    svm.fit(X_train, y_train)

    # Evaluate model
    precision, recall, f1 = evaluate_model(svm, X_test, y_test)
    return precision, recall, f1

# Train and evaluate SVM model using BoW features
precision_svm_bow, recall_svm_bow, f1_svm_bow = train_evaluate_svm(X_train_bow, y_train, X_test_bow, y_test)

# Train and evaluate SVM model using TF-IDF features
precision_svm_tfidf, recall_svm_tfidf, f1_svm_tfidf = train_evaluate_svm(X_train_tfidf, y_train, X_test_tfidf, y_test)

# Print the results
print(f'Precision Recall F1 (SVM, BoW): {precision_svm_bow:.2f} {recall_svm_bow:.2f} {f1_svm_bow:.2f}')
print(f'Precision Recall F1 (SVM, TF-IDF): {precision_svm_tfidf:.2f} {recall_svm_tfidf:.2f} {f1_svm_tfidf:.2f}')

Precision Recall F1 (SVM, BoW): 0.85 0.82 0.83
Precision Recall F1 (SVM, TF-IDF): 0.84 0.84 0.84
time: 48.9 s (started: 2023-09-12 13:07:44 +00:00)


## Logistic Regression Using Both Features

In [30]:
def train_evaluate_logistic_regression(X_train, y_train, X_test, y_test):
    # Initialize Logistic Regression model
    log_reg = LogisticRegression(max_iter=1000)

    # Train the model
    log_reg.fit(X_train, y_train)

    # Evaluate model
    precision, recall, f1 = evaluate_model(log_reg, X_test, y_test)

    return precision, recall, f1

# Train and evaluate Logistic Regression model using BoW features
precision_lr_bow, recall_lr_bow, f1_lr_bow = train_evaluate_logistic_regression(X_train_bow, y_train, X_test_bow, y_test)

# Train and evaluate Logistic Regression model using TF-IDF features
precision_lr_tfidf, recall_lr_tfidf, f1_lr_tfidf = train_evaluate_logistic_regression(X_train_tfidf, y_train, X_test_tfidf, y_test)

# Print the results
print(f'Precision Recall F1 (Logistic Regression, BoW): {precision_lr_bow:.2f} {recall_lr_bow:.2f} {f1_lr_bow:.2f}')
print(f'Precision Recall F1 (Logistic Regression, TF-IDF): {precision_lr_tfidf:.2f} {recall_lr_tfidf:.2f} {f1_lr_tfidf:.2f}')

Precision Recall F1 (Logistic Regression, BoW): 0.85 0.83 0.84
Precision Recall F1 (Logistic Regression, TF-IDF): 0.84 0.86 0.85
time: 5.82 s (started: 2023-09-12 12:53:04 +00:00)


## Naive Bayes Using Both Features

In [28]:
def train_evaluate_naive_bayes(X_train, y_train, X_test, y_test):
    # Initialize Naive Bayes model (Multinomial Naive Bayes for text classification)
    nb_model = MultinomialNB()

    # Train the model
    nb_model.fit(X_train, y_train)

    # Evaluate model
    precision, recall, f1 = evaluate_model(nb_model, X_test, y_test)

    return precision, recall, f1

# Train and evaluate Naive Bayes model using BoW features
precision_nb_bow, recall_nb_bow, f1_nb_bow = train_evaluate_naive_bayes(X_train_bow, y_train, X_test_bow, y_test)

# Train and evaluate Naive Bayes model using TF-IDF features
precision_nb_tfidf, recall_nb_tfidf, f1_nb_tfidf = train_evaluate_naive_bayes(X_train_tfidf, y_train, X_test_tfidf, y_test)

# Print the results
print(f'Precision Recall F1 (Naive Bayes, BoW): {precision_nb_bow:.2f} {recall_nb_bow:.2f} {f1_nb_bow:.2f}')
print(f'Precision Recall F1 (Naive Bayes, TF-IDF): {precision_nb_tfidf:.2f} {recall_nb_tfidf:.2f} {f1_nb_tfidf:.2f}')

Precision Recall F1 (Naive Bayes, BoW): 0.84 0.75 0.79
Precision Recall F1 (Naive Bayes, TF-IDF): 0.82 0.82 0.82
time: 285 ms (started: 2023-09-12 12:48:52 +00:00)


# THE END